In [12]:
import pandas as pd

In [20]:
import sqlite3
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


# Create db connection.
cnx = sqlite3.connect('data/database.sqlite')



In [21]:


seasons = ['2015/2016','2014/2015','2013/2014','2012/2013','2011/2012','2010/2011','2009/2010']
leagues_id = [1729]


In [27]:
df_matches = pd.read_sql_query("SELECT * FROM Match WHERE league_id = (?) AND season = (?)", cnx, params=(21518,season,)) 
df_matches['goal'][0]
df_matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,21898,21518,21518,2009/2010,1,2009-08-29 00:00:00,684585,8633,9783,3,2,1,2,4,6,8,4,6,3,5,7,5,1,2,4,6,8,4,6,3,5,7,5,1,3,3,3,3,6,6,8,8,8,11,1,3,3,3,3,6,6,8,8,8,11,30657.0,37442.0,34029.0,37581.0,28467.0,23988.0,30889.0,41044.0,30655.0,30893.0,26166.0,33018.0,111190.0,34104.0,37451.0,41167.0,21445.0,37449.0,37447.0,33761.0,31045.0,37446.0,<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>131</event_...,<shoton><value><stats><blocked>1</blocked></stats><event_incident_typefk>494</event_incident_typefk><elapsed>15</ela...,<shotoff><value><stats><shotoff>1</shotoff></stats><event_incident_typefk>46</event_incident_typefk><elapsed>2</elap...,<foulcommit><value><stats><foulscommitted>1</foulscommitted></stats><event_incident_typefk>37</event_incident_typefk...,<card><value><comment>y</comment><stats><ycards>1</ycards></stats><event_incident_typefk>73</event_incident_typefk><...,<cross><value><stats><crosses>1</crosses></stats><event_incident_typefk>7</event_incident_typefk><elapsed>7</elapsed...,<corner><value><stats><corners>1</corners></stats><event_incident_typefk>330</event_incident_typefk><elapsed>8</elap...,<possession><value><comment>59</comment><event_incident_typefk>352</event_incident_typefk><elapsed>25</elapsed><subt...,1.25,5.5,13.00,1.20,5.75,10.50,1.25,5.00,12.00,1.25,4.50,10.00,None,None,None,1.20,5.00,11.0,1.20,6.00,15.00,1.25,5.0,10.0,1.25,5.50,11.00,1.25,5.00,11.0
1,21899,21518,21518,2009/2010,1,2009-08-30 00:00:00,684586,8315,8558,1,0,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,4,6,3,5,7,5,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,6,6,8,8,8,11,33764.0,33025.0,37604.0,33027.0,74990.0,179772.0,37410.0,33024.0,181297.0,150832.0,33030.0,25563.0,164033.0,113318.0,38385.0,33767.0,24099.0,37441.0,24852.0,75194.0,30851.0,32764.0,<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>71</event_i...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycards>1</ycards></stats><event_incident_typefk>70</event_incident_typefk><...,<cross />,<corner />,<possession />,2.10,3.3,3.50,2.20,3.25,2.95,2.15,3.25,3.25,2.20,3.20,2.80,None,None,None,2.15,3.20,2.9,2.25,3.25,3.20,2.10,3.2,3.2,2.20,3.25,3.20,2.20,3.25,3.0
2,21900,21518,21518,2009/2010,1,2009-08-30 00:00:00,684587,8661,9868,2,0,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,4,6,3,5,7,5,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,6,6,8,8,8,11,37439.0,33856.0,33607.0,33849.0,33893.0,56686.0,37631.0,38160.0,33854.0,25462.0,33028.0,19435.0,75606.0,18388.0,38138.0,75609.0,75615.0,107930.0,102912.0,29904.0,37510.0,49672.0,<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>71</event

## Taking a look to the Match Table

First of all we filter just the games of the desire league and season

## Cleaning names of teams

In [4]:
import stringdist

# Normalize names 
def normalize_team_names(y):
    teams = []
    for t in list(y):
        tt = t.lower().split()
        rem = []
        cont=0
        #print(tt)
        for el in tt:
            cont +=1
            if el in ['athletico','atletico','athlético','atlético','atl','ath','athletic']: 
                if cont < len(tt):
                    tt[cont]='atl'+tt[cont]
            if el in ['manchester','man','sporting','sp','deportivo','la','de','real','fc','cf','ud','las','cd','balompié','rc','de','athletico','atletico','athlético','atlético','atl','ath','athletic']:
                rem.append(el)
            
                #print(tt)
        if rem != []:
            for r in rem:
                tt.remove(r)
        
        if len(tt)>1:
            if len(tt[0])>len(tt[1]):
                t = tt[0]
            else:
                t = tt[1]
        else:
            t=tt[0]
        #print(t)  
        teams.append(t)
    return teams

#Change name x for the most similar name in the teams_normalized list.
def func(x,teams_normalized):
    dist =0
    distmin=10
    xt = x.lower().split()
    rem = []
    cont=0
    for el in xt:
        cont +=1
        if el in ['athletico','atletico','athlético','atlético','atl','ath','athletic']:
            if cont < len(xt):
                xt[cont]='atl'+xt[cont]
        if el in ['manchester','man','sporting','sp','deportivo','la','de','real','fc','cf','ud','las','cd','balompié','rc','de','athletico','atletico','athlético','atlético','atl','ath','athletic']:
            rem.append(el)
    if rem != []:
        for r in rem:
                xt.remove(r)
                


    if len(xt)>1:
        if len(xt[0])>len(xt[1]):
            x = xt[0]
        else:
            x = xt[1]
    else:
        x=xt[0]

    
    for t in list(teams_normalized):
        dist = stringdist.levenshtein(x, t)
        #print(t)
        if dist < distmin:
            #print('................')
            #print(x)
            #print(t)
            #print(dist)
            #print('................')
            distmin = dist
            team =t
    return team



In [5]:
def convert_to(df,type_to_convert, all_categorical = False,columns = 'None'):
 
    if all_categorical == True:
        categoric_types = df.select_dtypes(include=['object'])
        if categoric_types.columns.empty:
            print('No object types in the dataframe to be converted')
            return df
        else:
            if 'labels' in categoric_types.columns:
                categoric_types = list(categoric_types.columns)
                categoric_types.remove('labels')
                df[categoric_types] = df[categoric_types].astype('category')
                columns = categoric_types
        
    else:
        df[columns] = df[columns].astype('category')
        
        
    if type_to_convert == 'labelling':
        for column in columns:
            df[column] = df[column].cat.codes
    elif type_to_convert == 'onehot':
        for column in columns:
            df= pd.get_dummies(df, columns=[column])             
      
    return df

In [6]:
def convert_to_standard_team_names(cnx, league_id, season):

    #.......... df_matches ..........
    
    #Read matches from the season and league specified.
    df_matches = pd.read_sql_query("SELECT * FROM Match WHERE league_id = (?) AND season = (?)", cnx, params=(league_id,season,)) 
    print(f'The shape of df_matches of league:{league_id} and season:{season} is: {df_matches.shape}')

    #Drop columns with not useful information (html text), we will get this information from another dataset from football-data.co.uk
    df_matches = df_matches.drop(columns=['goal', 'shoton', 'shotoff','foulcommit', 'card', 'cross', 'corner', 'possession'])

    #Convert types
    df_matches['date'] = pd.to_datetime(df_matches['date'])
    df_matches['date'] = pd.to_datetime(df_matches['date'],format='%Y%m%d')
    df_matches['stage'] = df_matches['stage'].astype(int)
    
   #To merge data with the other dataset we need a common team id between both datasets. We are going to create a unique string identifier for team names named team_name_id.
    
    
    #In the df_matches we will first include a column with the name of the team extracted from the Team table by means of the team_api_id.
    df_Teams = pd.read_sql_query("SELECT * FROM Team ", cnx)
    df_Teams.head()
    

    # Manually change some names due its difficulty to be treated for our string name procedure.
    df_Teams.loc[df_Teams['team_long_name']=='Athletic Club de Bilbao','team_long_name'] = 'Athletic Bilbao'
    df_Teams.loc[df_Teams['team_long_name']=='Real Sporting de Gijón','team_long_name'] = 'Sporting Gijon'
    
    df_Teams.loc[df_Teams['team_long_name']=='West Bromwich Albion','team_long_name'] = 'West Brom'
    df_Teams.loc[df_Teams['team_long_name']=='West Ham United','team_long_name'] = 'West Ham'
    df_Teams.loc[df_Teams['team_long_name']=='Queens Park Rangers','team_long_name'] = 'QPR'
    df_Teams.loc[df_Teams['team_long_name']=='Stoke City','team_long_name'] = 'Stoke'
    df_Teams.loc[df_Teams['team_long_name']=='Hull City','team_long_name'] = 'Hull'
    df_Teams.loc[df_Teams['team_long_name']=='Bolton Wanderers','team_long_name'] = 'Bolton'
    df_Teams.loc[df_Teams['team_long_name']=='Wolverhampton Wanderers', 'team_long_name'] = 'Wolves'
    
    
    
    
    #First we normalize the names of the teams
    teams_normalized = normalize_team_names(df_Teams['team_long_name'])
    
    #Now we normalize the names of df_Teams 
    df_Teams['team_name_id']= df_Teams['team_long_name'].apply(lambda x: func(x,teams_normalized))

    
    #Now we will have to include names of the teams into de df_matches dataframe, for this we have to merge df_matches with df_Teams on the "team_api_id".
    #Since df_matches just have names of the 'home_team_api_id' and 'away_team_api_id', we will add a new column referring the corresponding teams as 'home_team_name_id' and 'away_team_name_id'.
    df_matches['home_team_name_id']=df_matches.merge(df_Teams[['team_api_id','team_name_id']], left_on='home_team_api_id', right_on='team_api_id',how='left')['team_name_id']
    df_matches['away_team_name_id']=df_matches.merge(df_Teams[['team_api_id','team_name_id']], left_on='away_team_api_id', right_on='team_api_id',how='left')['team_name_id']

    #We finally have the df_matches with the Match Table information but with the home_tema_name_id and away_team_name_id which will be useful to create a common id between the two csv that we want to merge.
    
    #print(df_Teams)
    

    
    #.......... footballdata ..........
    
    #Read matches from the season and league specified.
    location_to_file = 'data/'
    season_footdata = season.replace('/','_').replace('20','') 
    df_footballdata = pd.read_csv(location_to_file+'{0}_{1}.csv'.format(league_id,season_footdata))   
    print(f'The shape of df_footballdata of league:{league_id} and season:{season_footdata} is: {df_footballdata.shape}')
  
    #Convert types    
    df_footballdata['date'] = pd.to_datetime(df_footballdata['Date'])
    df_footballdata['date'] = pd.to_datetime(df_footballdata['date'], format='%Y%m%d')

    
    #Creating same names for teams as the other dataset
    df_footballdata['home_team_name_id'] = df_footballdata['HomeTeam'].apply(lambda x: func(x,teams_normalized))
    df_footballdata['away_team_name_id'] = df_footballdata['AwayTeam'].apply(lambda x: func(x,teams_normalized))
    
    
    
    
    #Creating common match identifier for both datasets (uid)
    df_footballdata['uid']= df_footballdata['home_team_name_id']+df_footballdata['away_team_name_id']
    df_matches['uid']= df_matches['home_team_name_id']+df_matches['away_team_name_id']
    
    return df_matches, df_footballdata

In [7]:
def merge_matchtable_footballdata(season,league_id):
    
    df_matches, df_footballdata = convert_to_standard_team_names(cnx, league_id, season)
    
#     print((df_matches[['home_team_name_id','away_team_name_id','uid']]))
     
#     print((df_footballdata[['home_team_name_id','away_team_name_id']]))

#     print((df_matches['uid'].sort_values(ascending=True)))
#     print((df_footballdata['uid'].sort_values(ascending=True)))
    
    if len(set(df_matches['uid']).symmetric_difference(set(df_footballdata['uid']))) > 0:
        print('Names of uid of both datasets do not match.')
        print('CAUTION!!!! THE FOLLOWING NAMES HAVE DIFFERENT SPELLING IN THE TWO DATASETS: ',
              set(df_matches['uid']).symmetric_difference(set(df_footballdata['uid'])))
    
    # Merging both datasets
    df_complete_league_year = pd.merge(df_matches, df_footballdata[['FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY','AY', 'HR', 'AR','uid']], on='uid', how='inner')
    
    print(f'df_complete_league_year shape: {df_complete_league_year.shape}')

    
    
    # Implementing models with Real Data as the guys from Stanford suggest.
    # We create the dataset as it is stated at: 
    # *Shin, JongHo and Robert Gasparyan. “A novel way to Soccer Match Prediction.” (2014).*
    
    #Sort per date.
    df_complete_league_year['date'] = pd.to_datetime(df_complete_league_year['date'])
    df_complete_league_year=df_complete_league_year.sort_values(by='date')

    #We keep just the data need it for this.
    df_matches_info = df_complete_league_year[['id','uid','date','stage','home_team_name_id','away_team_name_id','FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY','AY', 'HR', 'AR']]  
    df_matches_info[['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF','HC', 'AC', 'HY','AY', 'HR', 'AR']] = df_matches_info.loc[:,['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF','HC', 'AC', 'HY','AY', 'HR', 'AR']].astype(float)

    
    #Creating a column for the labels
    labels = df_matches_info["FTR"]
    df_matches_info = pd.get_dummies(df_matches_info, columns=["FTR", "HTR"])
    df_matches_info['labels'] = labels

    #Creating the mean of each columns taking the past mathces (rows)
    df_stanford_model = df_matches_info.copy()
    df_stanford_model[['FTHG_mean','HTHG_mean','HS_mean','HST_mean','HF_mean','HC_mean','HY_mean','HR_mean','H_FTR_A_mean','H_FTR_D_mean','H_FTR_H_mean','H_HTR_A_mean','H_HTR_D_mean','H_HTR_H_mean']]= df_stanford_model.groupby('home_team_name_id')['FTHG','HTHG','HS','HST','HF','HC','HY','HR','FTR_A','FTR_D','FTR_H','HTR_A','HTR_D','HTR_H'].apply(lambda x:x.expanding().mean().shift())
    df_stanford_model[['FTAG_mean','HTAG_mean','AS_mean','AST_mean','AF_mean','AC_mean','AY_mean','AR_mean','A_FTR_A_mean','A_FTR_D_mean','A_FTR_H_mean','A_HTR_A_mean','A_HTR_D_mean','A_HTR_H_mean']]= df_stanford_model.groupby('away_team_name_id')['FTAG','HTAG','AS','AST','AF','AC','AY','AR','FTR_A','FTR_D','FTR_H','HTR_A','HTR_D','HTR_H'].apply(lambda x:x.expanding().mean().shift())

    #Selecting features
    df_stanford_real_features = df_stanford_model[['id','stage','date','home_team_name_id','away_team_name_id','labels','FTHG_mean','HTHG_mean','HS_mean','HST_mean','HF_mean','HC_mean','HY_mean','HR_mean','H_FTR_A_mean','H_FTR_D_mean','H_FTR_H_mean','H_HTR_A_mean','H_HTR_D_mean','H_HTR_H_mean','FTAG_mean','HTAG_mean','AS_mean','AST_mean','AF_mean','AC_mean','AY_mean','AR_mean','A_FTR_A_mean','A_FTR_D_mean','A_FTR_H_mean','A_HTR_A_mean','A_HTR_D_mean','A_HTR_H_mean']]

    #Explanation of name of columns
    pd.set_option('display.max_colwidth',120)
    exp = pd.DataFrame({'abbreviations':df_stanford_real_features.columns,'Explanation':['id of the match','stage','date','home_team_name_id','away_team_name_id','labels','Mean of Full Time Home Team Goals','Mean of Half Time Home Team Goals','Mean of Home Team Shots','Mean of Home Team Shots on Target','Mean of Home Team Fouls Committed','Mean of Home Team Corners','Mean of Home Team Yellow Cards','Mean of Home Team Red Cards','Mean of games lost at home for the Home Team','Mean of games draw at home for the Home Team','Mean of games won at home for the Home Team ','Mean of games lost at Half Time at home for the Home Team','Mean of games draw at Half Time at home for the Home Team','Mean of games won at Half Time at home for the Home Team','Mean of Full Time Away Team Goals','Mean of Half Time Away Team Goals','Mean of Away Team Shots','Mean of Away Team Shots on Target','Mean of Away Team Fouls Committed','Mean of Away Team Corners','Mean of Away Team Yellow Cards','Mean of Away Team Red Cards','Mean of games won away for the Away Team','Mean of games draw away for the Away Team','Mean of games lost Away for the Away Team ','Mean of games won at Half Time away for the Away Team','Mean of games draw at Half Time away for the Away Team','Mean of games lost at Half Time away for the Away Team']})
    #print(exp)
    
    #Save df in csv
    season_footdata = season.replace('/','_').replace('20','') 
    df_stanford_real_features.to_csv(f'data/df_stanford_real_features_leagues_{league_id}_seasons_{season_footdata}.csv',index=False)

    
    return df_stanford_real_features


In [8]:
datasets = []
for league_id in leagues_id:
    for season in seasons:
        dataset_season = merge_matchtable_footballdata(season,league_id)
        datasets.append(dataset_season.loc[:, :])
        
df_stanford_real_features_ALL_SEASONS = pd.concat(datasets)

The shape of df_matches of league:1729 and season:2015/2016 is: (380, 115)
The shape of df_footballdata of league:1729 and season:15_16 is: (380, 65)
df_complete_league_year shape: (380, 128)


/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


The shape of df_matches of league:1729 and season:2014/2015 is: (380, 115)
The shape of df_footballdata of league:1729 and season:14_15 is: (380, 68)
df_complete_league_year shape: (380, 128)
The shape of df_matches of league:1729 and season:2013/2014 is: (380, 115)
The shape of df_footballdata of league:1729 and season:13_14 is: (380, 68)
df_complete_league_year shape: (380, 128)
The shape of df_matches of league:1729 and season:2012/2013 is: (380, 115)
The shape of df_footballdata of league:1729 and season:12_13 is: (380, 74)
df_complete_league_year shape: (380, 128)
The shape of df_matches of league:1729 and season:2011/2012 is: (380, 115)
The shape of df_footballdata of league:1729 and season:11_12 is: (380, 71)
df_complete_league_year shape: (380, 128)
The shape of df_matches of league:1729 and season:2010/2011 is: (380, 115)
The shape of df_footballdata of league:1729 and season:10_11 is: (380, 71)
df_complete_league_year shape: (380, 128)
The shape of df_matches of league:1729 a

In [9]:
seasons_data = []
for season in seasons:
    seasons_data.append(season.replace('/','_').replace('20',''))

location_to_file = 'data/'
df_stanford_real_features_ALL_SEASONS.to_csv(location_to_file+f'df_stanford_real_features_leagues_{leagues_id}_seasons_{seasons_data}.csv',index=False)

In [10]:
df_stanford_real_features_ALL_SEASONS.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2660 entries, 1 to 311
Data columns (total 34 columns):
id                   2660 non-null int64
stage                2660 non-null int64
date                 2660 non-null datetime64[ns]
home_team_name_id    2660 non-null object
away_team_name_id    2660 non-null object
labels               2660 non-null object
FTHG_mean            2520 non-null float64
HTHG_mean            2520 non-null float64
HS_mean              2520 non-null float64
HST_mean             2520 non-null float64
HF_mean              2520 non-null float64
HC_mean              2520 non-null float64
HY_mean              2520 non-null float64
HR_mean              2520 non-null float64
H_FTR_A_mean         2520 non-null float64
H_FTR_D_mean         2520 non-null float64
H_FTR_H_mean         2520 non-null float64
H_HTR_A_mean         2520 non-null float64
H_HTR_D_mean         2520 non-null float64
H_HTR_H_mean         2520 non-null float64
FTAG_mean            2520 non-null

In [11]:
df_player_attributes = pd.read_sql_query("SELECT * FROM Match WHERE season ='2011/2012' and league_id ", cnx)
df_player_attributes

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,757,1,1,2011/2012,1,2011-07-29 00:00:00,1032692,1773,8635,2,1,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,3.0,4.0,7.0,1.0,3.0,5.0,7.0,9.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,1.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,10.0,10.0,37993.0,37865.0,37051.0,45840.0,179059.0,37981.0,38791.0,37963.0,38777.0,45865.0,68114.0,38391.0,38389.0,208493.0,149150.0,40536.0,38253.0,114333.0,178249.0,265123.0,46552.0,181276.0,None,None,None,None,None,None,None,None,7.00,4.00,1.50,6.50,4.10,1.42,5.00,3.7,1.50,5.50,3.60,1.50,None,None,None,5.50,3.75,1.57,6.00,3.60,1.57,7.000,4.00,1.500,6.00,3.75,1.50,6.50,4.00,1.44
1,758,1,1,2011/2012,1,2011-07-30 00:00:00,1032693,9998,9985,1,1,1.0,1.0,3.0,5.0,7.0,9.0,3.0,4.0,8.0,6.0,5.0,1.0,2.0,4.0,6.0,8.0,6.0,5.0,4.0,3.0,5.0,7.0,1.0,3.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,8.0,6.0,10.0,10.0,10.0,39153.0,39977.0,181574.0,166577.0,38906.0,36849.0,21753.0,37128.0,149258.0,26224.0,78902.0,38797.0,129462.0,245653.0,164229.0,33620.0,38969.0,17276.0,119117.0,38382.0,248689.0,46335.0,None,None,None,None,None,None,None,None,5.00,3.50,1.73,4.75,3.50,1.65,3.80,3.3,1.75,4.50,3.50,1.62,None,None,None,4.33,3.50,1.75,4.50,3.50,1.73,4.750,3.60,1.750,4.50,3.40,1.72,4.50,3.60,1.67
2,759,1,1,2011/2012,1,2011-07-30 00:00:00,1032694,9987,9993,3,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,91929.0,94462.0,38368.0,148314.0,109331.0,104411.0,39498.0,169200.0,43158.0,42153.0,38794.0,36873.0,57078.0,38800.0,174363.0,27508.0,38784.0,163613.0,38371.0,33622.0,166679.0,14487.0,None,None,None,None,None,None,None,None,1.44,4.33,7.00,1.45,3.95,6.25,1.50,3.7,5.00,1.44,3.75,6.00,None,None,None,1.40,4.33,7.00,1.44,4.20,6.50,1.440,4.50,7.000,1.45,4.00,6.25,1.44,4.00,6.50
3,760,1,1,2011/2012,1,2011-07-30 00:00:00,1032695,9991,9984,0,1,1.0,1.0,3.0,5.0,7.0,9.0,3.0,5.0,7.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,3.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,37854.0,12473.0,114368.0,178096.0,37440.0,NaN,33662.0,26771.0,166618.0,148329.0,12574.0,36835.0,38342.0,243250.0,37047.0,38789.0,27110.0,166670.0,188231.0,277766.0,38251.0,209855.0,None,None,None,None,None,None,None,None,1.57,3.80,6.00,1.55,3.85,5.00,1.55,3.5,4.80,1.57,3.50,5.00,None,None,None,1.60,3.75,5.00,1.50,3.90,6.00,1.530,4.00,6.000,1.55,3.75,5.50,1.53,3.75,5.50
4,761,1,1,2011/2012,1,2011-07-30 00:00:00,1032696,9994,10000,0,0,1.0,3.0,4.0,7.0,1.0,3.0,5.0,7.0,9.0,4.0,6.0,1.0,1.0,3.0,5.0,7.0,9.0,3.0,5.0,7.0,4.0,6.0,1.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,